<img src = "https://i.imgur.com/UjutVJd.jpg" align = "center">

## Membangun Jaringan Saraf Tiruan

In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

np.set_printoptions(precision=7)

### Single Layer Perceptron
Seperti yang sudah kalian pelajari, di dalam Jaringan Saraf Tiruan, pengetahuan yang dipelajari disimpan dalam bobot masing-masing neuron. Untuk mempermudah perhitungan, neuron dikumpulkan ke dalam kelompok-kelompok yang disebut layer sehingga bobot-bobot neuron dalam satu layer dapat dibentuk sebagai sebuah matrik.

Sebagai contoh, misalkan kita hanya menggunakan satu layer saja, atau yang biasa disebut sebagai ***Single-Layer Perceptron***, maka jaringan akan terlihat seperti ilustrasi di bawah.

![onelayer](https://image.ibb.co/fjR3oz/onelayer.png) 


Dari ilustrasi dapat dilihat bahwa Jaringan Saraf Tiruan dengan hanya satu layer tidak lain adalah fungsi regresi linier. 

Sudah kalian ketahui juga bahwa JST satu layer dengan pelatihan *gradient descent* yang terjadi adalah
* melakukan proses maju yang mengalikan input dengan bobot,
* kemudian melakukan proses mundur untuk mendapatkan gradien input dan gradien bobot

Jika kita tuliskan secara singkat, kode Single Layer Perceptron dalam bahasa python, kita hanya membutuhkan beberapa baris
```python
for epoch in range(max_epoch):
    
    layer = np.dot(fitur, bobot0)+bias0
    aktivasi = 1 / (1 + np.exp(-layer))    

    error = target - aktivasi

    g_aktivasi = (err) * (aktivasi * (1 - aktivasi))
    g_bobot0 = fitur.T.dot(layer)
    
    bobot0 = bobot0 + lr*g_bobot
    
```


### Multi Layer Perceptron
Dalam penggunaan Jaringan Saraf Tiruan, kita dapat menumpuk layer-layer neuron menjadi sebuah arsitektur baru yang disebut sebagai *Multi-layer Perceptron* (MLP). Layer-layer yang berada di antara layer input dan layer output disebut sebagai *hidden layer*

MLP dengan 2 layer neuron biasa disebut sebagai *2-layer neural net* atau *1-hidden-layer neural net*.
begitu pula untuk MLP dengan 3 layer biasa disebut sebagai *3-layer neural net* atau *2-hidden-layer neural net*. Seperti ilustrasi berikut

*2-layer NN* | *3-layer NN*
- | -
![2layerNN](https://image.ibb.co/dHnnFe/2layerNN.png) | ![3layerNN](https://image.ibb.co/iH18MK/3layerNN.png)

Di sini dapat kalian perhatikan bahwa sebenarnya, selama pelatihan, MLP mengulang-ulang proses yang dilakukan Single Layer Perceptron, baik pada proses maju dan mundur. Kita bisa saja menuliskan proses pelatihan yang spesifik untuk setiap model kedalaman jaringan.

Namun alangkah mudahnya jika kita bisa menuliskannya dalam suatu bentuk fungsi sehingga kita menjadi mudah menambah atau mengurangi jumlah kedalaman model

**Inilah yang mendasari dibangunnya library pembelajaran mesin yang populer seperti Theano, TensorFlow, dan Torch**
---

## API Jaringan Saraf Tiruan

---
### a. Forward and Backward Affine Function

Mula-mula mari kita implementasikan kembali perhitungan maju untuk sebuah layer ke dalam sebuah fungsi

$$
\begin{align}
f(x, W, b) = x.W + b
\end{align}
$$


In [0]:
def affine_forward(x, W, b):   
    v = np.dot(x, W) + b # x dot w + b
    
    cache = (x, W, b)
    
    return v, cache

Dilanjut dengan menuliskan fungsi untuk proses perhitungan mundur


$$
\begin{align*}
\partial W & = x^T.\partial out \\
\partial b & = \sum \partial out \\
\partial x & = \partial out.W^T \\
\end{align*}
$$

In [0]:
def affine_backward(dout, cache):
    
    x, W, b = cache
    
    dW = np.dot(x.T, dout) # x' dot dout
    
    db = np.sum(dout, axis=0, keepdims=True)
    
    dx = np.dot(dout, W.T) # dout dot W'
    
    return dW, db, dx

---
### b. Forward and Backward Sigmoid Function

Jangan lupa, implementasikan juga fungsi proses maju untuk fungsi aktivasi Sigmoid

$$
\begin{align}
f(x) = \sigma(x) = \frac{1}{1+e^{-v}}
\end{align}
$$


In [0]:
def sigmoid_forward(x):  
    
    out =  1 / ( 1 + np.exp(-x) ) 
    
    return out  

---
Dan juga fungsi proses mundur untuk fungsi aktivasi Sigmoid
$$
\begin{align*}
\sigma'(x) = \sigma(x) \ (1 - \sigma(x))\tag{2}\\\\
\partial out = \partial out . \sigma'(x)
\end{align*}
$$
<br>

In [0]:
def sigmoid_backward(dout, ds):
    """
    Argument:
        ds: sigmoid forward result
        dout: gradient error
    """
    ds_ = ds * (1 - ds)
    
    dout = dout * ds_
    
    return dout

## Implementasi JST dengan API

sekarang untuk membangun JST Single Layer Perceptron, dalam satu epoch, kita tinggal menyusun fungsi
<pre><font color='green'>affine_fwd</font> -> 
    <font color='blue'>sigmoid_fwd</font> ->
        <font color='red'>hitung_error</font> ->
    <font color='blue'>sigmoid_bwd</font> -> 
<font color='green'>affine_bwd</font> -> 
update_bobot</pre>

---

Kemudian untuk membangun JST 2 Layer (1 hidden layer), susunan fungsi di dalam setiap epoch menjadi
<pre><font color='green'>affine_fwd</font> -> 
    <font color='blue'>sigmoid_fwd</font> ->
        <font color='green'>affine_fwd</font> -> 
            <font color='blue'>sigmoid_fwd</font> ->
                <font color='red'>hitung_error</font> ->
            <font color='blue'>sigmoid_bwd</font> -> 
        <font color='green'>affine_bwd</font> -> 
    <font color='blue'>sigmoid_bwd</font> -> 
<font color='green'>affine_bwd</font> -> 
update_bobot</pre>


Sekarang, mari kita coba implementasikan

### SIngle Layer Perceptron
Sekarang, setelah kita memiliki API layer-layernya, untuk membangun jaringan menjadi sangat mudah.

Misalkan kita memiliki data sederhana sebagai berikut

In [38]:
x = np.array([[0, 0, 1], [0, 1, 1], [1, 0, 1], [1, 1, 1]])
y = np.array([[0, 1, 1, 0]]).T

print(x.shape)
print(y.shape)

nfitur = x.shape[1]   # 3 fitur
nlabel = y.shape[1]   # 1 bit label

(4, 3)
(4, 1)


Jika kita ingin melatih Single Layer Perceptron, maka kita hanya perlu menuliskan sebagai berikut

In [19]:
# inisialisasi bobot
w0 = 2*np.random.random((nfitur, nlabel)) - 1
b0 = np.zeros((1, nlabel))

print(w0.shape)

(3, 1)


In [29]:
for i in range(2000):
    # proses maju
    layer1, cache1 = affine_forward(x, w0, b0)
    aktivasi1 = sigmoid_forward(layer1)

    # hitung error
    error = y - aktivasi1
    print("mse = %0.20f" % (np.mean(error ** 2)))

    #proses mundur
    g_layer1 = sigmoid_backward(error, aktivasi1)
    dw0, db0, dx = affine_backward(g_layer1, cache1)

    #update bobot
    lr = 0.2
    w0 += lr * dw0
    b0 += lr * db0

mse = 0.25000000000000000000
mse = 0.24999999999999997224
mse = 0.25000000000000000000
mse = 0.24999999999999997224
mse = 0.24999999999999997224
mse = 0.24999999999999997224
mse = 0.25000000000000000000
mse = 0.25000000000000000000
mse = 0.25000000000000000000
mse = 0.25000000000000000000
mse = 0.25000000000000000000
mse = 0.25000000000000000000
mse = 0.25000000000000000000
mse = 0.24999999999999997224
mse = 0.24999999999999994449
mse = 0.24999999999999994449
mse = 0.24999999999999997224
mse = 0.24999999999999997224
mse = 0.25000000000000000000
mse = 0.25000000000000000000
mse = 0.25000000000000000000
mse = 0.25000000000000000000
mse = 0.25000000000000000000
mse = 0.25000000000000000000
mse = 0.25000000000000000000
mse = 0.25000000000000000000
mse = 0.25000000000000000000
mse = 0.24999999999999997224
mse = 0.24999999999999997224
mse = 0.24999999999999997224
mse = 0.24999999999999997224
mse = 0.25000000000000000000
mse = 0.24999999999999997224
mse = 0.24999999999999997224
mse = 0.250000

Kode dalam cell di atas menjalankan proses pelatihan 1 epoch, 

ulagi menjalankan cell tersebut untuk melanjutkan pelatihan

### Multi Layer Perceptron
Jika kita ingin melatih Multi Layer Perceptron, misalnya 2 layer, maka kita hanya perlu menambahkan

In [36]:
# inisialisasi bobot
nhidden = 10
w0 = 2*np.random.random((nfitur, nhidden)) -1
w1 = 2*np.random.random((nhidden, nlabel)) -1
print(w0.shape)
print(w1.shape)
b0 = np.zeros((1, nhidden))
b1 = np.zeros((1, nlabel))

(3, 10)
(10, 1)


In [0]:
for i in range(1000):
    # proses maju
    layer1, cache1 = affine_forward(x, w0, b0)
    aktivasi1 = sigmoid_forward(layer1)

    layer2, cache2 = affine_forward(aktivasi1, w1, b1)
    aktivasi2 = sigmoid_forward(layer2)


    # hitung error
    error = y - aktivasi2
    print("mse = %0.7f" % (np.mean(error ** 2)))


    #proses mundur
    g_layer2 = sigmoid_backward(error, aktivasi2)
    dw1, db1, g_aktivasi1 = affine_backward(g_layer2, cache2)

    g_layer1 = sigmoid_backward(g_aktivasi1, aktivasi1)
    dw0, db0, dx = affine_backward(g_layer1, cache1)



    #update bobot
    lr = 0.2
    w1 += lr * dw1
    w0 += lr * dw0
    b1 += lr * db1
    b0 += lr * db0

Kode dalam cell di atas menjalankan proses pelatihan 1 epoch, 

ulagi menjalankan cell tersebut untuk melanjutkan pelatihan

## Kesimpulan dan Penutup
* Pembangunan model Jaringan Saraf Tiruan dapat dipermudah dengan membangun fungsi API
* Cobalah mengubah kode di atas menjadi JST 3 layer

# Three Layer

In [0]:
# inisialisasi bobot
nhidden = 10
w0 = 2*np.random.random((nfitur, nhidden)) -1
w1 = 2*np.random.random((nhidden, nlabel)) -1

b0 = np.zeros((1, nhidden))
b1 = np.zeros((1, nlabel))

<p>Copyright &copy; 2019 ADF </p>